In [1]:
from datasets import load_dataset, Dataset
from vllm import LLM, SamplingParams

In [2]:
NUM_GPUS = 4
BATCH_SIZE = 2048

In [3]:
#ds = load_dataset('allenai/real-toxicity-prompts', split='train')
#ds = load_dataset('amang1802/wildeweb-sample-realtoxicity-challenge', split='train')


In [4]:
ds = ds.filter(lambda row: row['challenging'])

In [5]:
ds

Dataset({
    features: ['filename', 'begin', 'end', 'challenging', 'prompt', 'continuation', 'wildeweb'],
    num_rows: 770
})

In [6]:
model_id = "meta-llama/Llama-3.1-70B"

In [7]:
llm = LLM(model=model_id, max_model_len=512, tensor_parallel_size=NUM_GPUS, gpu_memory_utilization=0.98)

INFO 01-21 01:33:21 config.py:905] Defaulting to use mp for distributed inference
INFO 01-21 01:33:21 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Llama-3.1-70B', speculative_config=None, tokenizer='meta-llama/Llama-3.1-70B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=512, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Llama-3.1-70B, num_schedul

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


INFO 01-21 01:33:38 model_runner.py:1067] Loading model weights took 32.8892 GB
(VllmWorkerProcess pid=55391) INFO 01-21 01:33:38 model_runner.py:1067] Loading model weights took 32.8892 GB
(VllmWorkerProcess pid=55393) INFO 01-21 01:33:38 model_runner.py:1067] Loading model weights took 32.8892 GB
(VllmWorkerProcess pid=55392) INFO 01-21 01:33:39 model_runner.py:1067] Loading model weights took 32.8892 GB
INFO 01-21 01:33:40 distributed_gpu_executor.py:57] # GPU blocks: 79201, # CPU blocks: 3276
INFO 01-21 01:33:40 distributed_gpu_executor.py:61] Maximum concurrency for 512 tokens per request: 2475.03x
INFO 01-21 01:33:42 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-21 01:33:42 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decrea

In [9]:
def complete(jsons, output_column):
    prompts = [item['text'] for item in jsons]
    outputs = llm.generate(prompts, SamplingParams(temperature=0.5, top_p=0.9, max_tokens=64, stop=["\n"]))

    return {output_column: [output.outputs[0].text for output in outputs]}

In [10]:
ds_completions = ds.map(lambda prompts: complete(prompts, "metallama"), input_columns=['prompt'], batched=True, batch_size=BATCH_SIZE)

Parameter 'function'=<function <lambda> at 0x7486fc2740d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/770 [00:00<?, ? examples/s]


cessed prompts:   0% 0/770 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
cessed prompts:   0% 1/770 [00:00<04:05,  3.14it/s, est. speed input: 53.32 toks/s, output: 3.14 toks/s]
cessed prompts:   1% 5/770 [00:00<01:06, 11.43it/s, est. speed input: 153.89 toks/s, output: 11.84 toks/s]
cessed prompts:   3% 21/770 [00:00<00:15, 48.94it/s, est. speed input: 507.20 toks/s, output: 71.76 toks/s]
cessed prompts:   4% 29/770 [00:00<00:14, 51.27it/s, est. speed input: 606.58 toks/s, output: 103.08 toks/s]
cessed prompts:   5% 38/770 [00:00<00:13, 55.13it/s, est. speed input: 653.80 toks/s, output: 142.32 toks/s]
cessed prompts:   6% 47/770 [00:01<00:12, 57.69it/s, est. speed input: 684.36 toks/s, output: 188.12 toks/s]
cessed prompts:   8% 61/770 [00:01<00:09, 76.46it/s, est. speed input: 854.56 toks/s, output: 263.95 toks/s]
cessed prompts:   9% 70/770 [00:01<00:09, 72.83it/s, est. speed input: 868.47 toks/s, output: 300.89 toks/s]
cessed prompts:  10% 79/770 [00:01<00:

In [11]:
ds_completions.push_to_hub('amang1802/wildeweb-sample-realtoxicity-challenge')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/datasets/amang1802/wildeweb-sample-realtoxicity-challenge/commit/d389ca144cf5cc960e63905c366a4e6dc22c4a49', commit_message='Upload dataset', commit_description='', oid='d389ca144cf5cc960e63905c366a4e6dc22c4a49', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/amang1802/wildeweb-sample-realtoxicity-challenge', endpoint='https://huggingface.co', repo_type='dataset', repo_id='amang1802/wildeweb-sample-realtoxicity-challenge'), pr_revision=None, pr_num=None)